In [ ]:
try:
  import cudaq
  import cudaq_solvers as solvers
  import dynamiqs as dq
except(ImportError):
  %pip install cudaq
  %pip install cudaq-solvers
  %pip install dynamiqs

  import cudaq
  import cudaq_solvers as solvers
  import dynamiqs as dq

import numpy as np
from scipy.optimize import minimize
import jax.numpy as jnp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 5.3 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.12.0.post1-py3-none-any.whl size=7140 sha256=ed2a5ad400240f3b38e027c93dcf36fda5f266021e7283a6613a917a6580d31e
  Stored in directory: /root/.cache/pip/wheels/28/46/73/06c85cbac5fac9d1aef5f8ffd2e96d1acda5bd101fdce3c0b9
Successfully built cudaq
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Attempting to convert 2x2 bosonic annihilation/creation operators to linear combination of pauli matrices using the following formula:

𝐚 = $\frac{1}{2}$\($\sigma_x+i\sigma_y$\)

𝐚† = $\frac{1}{2}$\($\sigma_x-i\sigma_y$\)



In [ ]:
def sig_min(q):
  return 0.5 * (cudaq.spin.x(q) - 1j * cudaq.spin.y(q))
def sig_plus(q):
  return 0.5 * (cudaq.spin.x(q) + 1j * cudaq.spin.y(q))


Using the previously defined nnihilation and creation operators to model the cat qubit Hamiltonian

The Hamiltonian of the system is given by
$$
\begin{aligned}
H &= H_{\mathrm{2ph}} + H_d,\qquad \mathrm{with}\\
H_{\mathrm{2ph}} &= g_2 {a^\dagger}^2 b + g_2^* a^2 b^\dagger,\\
\hat{H}_d &= \epsilon_d^* b + \epsilon_d b^\dagger.
\end{aligned}
$$

Here, $\hat{H}_{\mathrm{2ph}}$ is the two-photon exchane Hamiltonian and $\hat{H}_d$ is the buffer drive Hamiltonian.g

Here we are using a two qubit system, one qubit for the memory mode, and one for the buffer mode. Therefore, we are defining 𝐚 as the annihilation operator operating on qubit 0, and 𝐛 as the annihilation acting on qubit 1

In [ ]:
def create_H(g2=1.0, eps_d=-4):

  a = sig_plus(0)
  a_dag = sig_min(0)

  b = sig_plus(1)
  b_dag = sig_min(1)

  H_2ph = g2*(a*a*b_dag + a_dag*a_dag*b)

  H_d = eps_d * (b+b_dag)

  H = H_2ph + H_d

  return H

def creat_H_matrix(g2=1,eps_d=-4,na=2,nb=2):

  a,b = dq.destroy(na,nb)

  H_2ph = g2*(a@a@b.dag() + a.dag()@a.dag()@b)
  H_d = eps_d * (b+b.dag())
  H = H_2ph + H_d

  return H


In [ ]:


def ansatz(thetas: list[float], n: int) -> cudaq.Kernel:
    # Create a CUDA-Q parameterized kernel
    paramterized_ansatz, variational_angles = cudaq.make_kernel(list)
    qubits = paramterized_ansatz.qalloc(n)

    # Using |+> as the initial state:
    paramterized_ansatz.h(qubits[0])
    paramterized_ansatz.cx(qubits[0], qubits[1])

    paramterized_ansatz.rx(variational_angles[0], qubits[0])
    paramterized_ansatz.cx(qubits[0], qubits[1])
    paramterized_ansatz.rz(variational_angles[1], qubits[1])
    paramterized_ansatz.cx(qubits[0], qubits[1])
    return paramterized_ansatz

In [ ]:
def run_logical_vqe(cudaq_hamiltonian: cudaq.SpinOperator) -> tuple[float, list[float]]:
    # Set seed for easier reproduction
    rng = np.random.default_rng(42)

    # Initial angles for the optimizer
    init_angles = rng.random(2) * 1e-1

    # Obtain CUDA-Q Ansatz
    num_qubits = cudaq_hamiltonian.get_qubit_count()
    variational_kernel = ansatz(num_qubits)

    # Perform VQE optimization
    energy, params, _ = solvers.vqe(
        variational_kernel,
        cudaq_hamiltonian,
        init_angles,
        optimizer=minimize,
        method="SLSQP",
        tol=1e-10,
    )
    return energy, params

In [ ]:
def run_vqe_shutter(cudaq_hamiltonian: cudaq.SpinOperator) -> tuple[float, list[float]]:
    # Set seed for easier reproduction
    rng = np.random.default_rng(42)

    # Initial angles for the optimizer
    init_angles = rng.random(2) * 1e-1

    # Obtain CUDA-Q Ansatz
    num_qubits = cudaq_hamiltonian.get_qubit_count()
    variational_kernel = ansatz(num_qubits)

    energies =[]
    thetas = []


    def iteration_callback(out):

      print(out)

      fxk = out[0]
      xk = out[1]

      energies.append(fxk)
      thetas.append(xk)
      #print(f"Iteration: {len(energies)} | theta = {xk} | energy = {fxk}")

    def callback(xk):
      exp_val = cudaq.observe(ansatz, H, xk[0]).expectation()
      print(f"Energy at iteration: {exp_val}")

    # Perform VQE optimization
    energy, params, _ = solvers.vqe(
      variational_kernel,
      cudaq_hamiltonian,
      init_angles,
      optimizer=minimize,
      method="SLSQP",
      tol=1e-10,
      callback = callback
    )
    return energy, params, energies, thetas

In [ ]:
import cudaq
from cudaq import spin
import cudaq_solvers as solvers



rng = np.random.default_rng(42)
init_angles = rng.random(2) * 1e-1



# Define quantum kernel (ansatz)
@cudaq.kernel
def ansatz(thetas: list[float]):
    q = cudaq.qvector(2)
    h(q[0])
    x.ctrl(q[0], q[1])

    rx(thetas[0],q[0])
    x.ctrl(q[0], q[1])
    rz(thetas[1], q[1])
    x.ctrl(q[0], q[1])

def callback(xk):
    exp_val = cudaq.observe(ansatz, H, xk).expectation()
    print(f"Energy at iteration: {exp_val}")

# Define Hamiltonian
H = create_H()

from scipy.optimize import minimize

# Run VQE with defaults (cobyla optimizer)
energy, parameters, data = solvers.vqe(
    lambda thetas: ansatz(thetas),
    H,
    initial_parameters=init_angles,
    optimizer=minimize,
    callback=callback,
    method='L-BFGS-B',
    verbose=True,
    jac='3-point',
    tol=1e-4,
    options={'disp': True}
)
print(f"Ground state energy: {energy}")

Energy at iteration: -3.4233804571903477
Energy at iteration: -3.4342689813212823
Energy at iteration: -3.6783547167130792
Energy at iteration: -3.9873663557515275
Energy at iteration: -3.999975219584801
Energy at iteration: -3.99999999981006
Ground state energy: -3.9999999999997464


/tmp/ipython-input-690960108.py:34: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  energy, parameters, data = solvers.vqe(


In [ ]:
# Using L-BFGS-B optimizer with parameter-shift gradients
energy, parameters, data = solvers.vqe(
    lambda thetas: ansatz(thetas),
    H,
    initial_parameters=[0.3,0.6],
    optimizer='lbfgs',
    gradient='parameter_shift',
    verbose=True
)

# Using SciPy optimizer directly
from scipy.optimize import minimize



energy, parameters, data = solvers.vqe(
    lambda thetas: ansatz(thetas),
    H,
    initial_parameters=[0.0,0.6],
    optimizer=minimize,
    callback=callback,
    method='L-BFGS-B',
    jac='3-point',
    tol=1e-4,
    options={'disp': True}
)

In [ ]:
cudaq.register_operation("meas_id", np.identity(2))

In [ ]:
def aim_logical_circuit(
    angles: list[float], basis: str, *, ignore_meas_id: bool = False
) -> cudaq.Kernel:
    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(6)

    kernel.for_loop(start=0, stop=3, function=lambda idx: kernel.h(qubits[idx]))
    kernel.cx(qubits[1], qubits[4])
    kernel.cx(qubits[2], qubits[3])
    kernel.cx(qubits[0], qubits[1])
    kernel.cx(qubits[0], qubits[3])

    # Rx teleportation
    kernel.rx(angles[0], qubits[0])

    kernel.cx(qubits[0], qubits[1])
    kernel.cx(qubits[0], qubits[3])
    kernel.h(qubits[0])

    if basis == "z_basis":
        if not ignore_meas_id:
            kernel.for_loop(
                start=0,
                stop=5,
                function=lambda idx: getattr(kernel, "meas_id")(qubits[idx]),  # noqa: B009
            )
        kernel.mz(qubits)
    elif basis == "x_basis":
        # ZZ rotation and teleportation
        kernel.cx(qubits[3], qubits[5])
        kernel.cx(qubits[2], qubits[5])
        kernel.rz(angles[1], qubits[5])
        kernel.cx(qubits[1], qubits[5])
        kernel.cx(qubits[4], qubits[5])
        kernel.for_loop(start=1, stop=5, function=lambda idx: kernel.h(qubits[idx]))
        if not ignore_meas_id:
            kernel.for_loop(
                start=0,
                stop=6,
                function=lambda idx: getattr(kernel, "meas_id")(qubits[idx]),  # noqa: B009
            )
        kernel.mz(qubits)
    else:
        raise ValueError("Unsupported basis provided:", basis)
    return kernel

In [ ]:
def generate_circuit_set(ignore_meas_id: bool = False) -> object:

    H = create_H()
    energy, opt_params, energies, thetas = run_vqe_shutter(H)
    angles = [float(angle) for angle in opt_params]

    print(energy)

    kernel = cudaq.make_kernel()
    qubits = kernel.qalloc(2)

    kernel.h(qubits[0])
    kernel.cx(qubits[0], qubits[1])

    kernel.rx(angles[0], qubits[0])
    kernel.cx(qubits[0], qubits[1])
    kernel.rz(angles[1], qubits[1])
    kernel.cx(qubits[0], qubits[1])


    print(cudaq.draw(kernel))









    print("\nFinished building optimized circuits!")

In [ ]:
sim_circuit_dict = generate_circuit_set()

/tmp/ipython-input-5598012.py:9: DeprecationWarning: use `qubit_count` instead
  num_qubits = cudaq_hamiltonian.get_qubit_count()
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



RuntimeError: error: Invalid runtime argument type. Argument of type <class 'int'> was provided, but list[float] was expected.

Offending code:
  File "<frozen runpy>", line 198, in _run_module_as_main


During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
RuntimeError: error: Invalid runtime argument type. Argument of type <class 'int'> was provided, but list[float] was expected.

Offending code:
  File "<frozen runpy>", line 198, in _run_module_as_main


During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

AttributeError: 'TypeError'

In [ ]:
paramterized_ansatz, variational_angles = cudaq.make_kernel(list)

In [ ]:
print(paramterized_ansatz)

module attributes {quake.mangled_name_map = {__nvqpp__mlirgen____nvqppBuilderKernel_XB47179ITS = "__nvqpp__mlirgen____nvqppBuilderKernel_XB47179ITS_PyKernelEntryPointRewrite"}} {
  func.func @__nvqpp__mlirgen____nvqppBuilderKernel_XB47179ITS(%arg0: !cc.stdvec<f64>) attributes {"cudaq-entrypoint", "cudaq-kernel"} {
    return
  }
}



In [ ]:
print(variational_angles[7])

Value(%8 = cc.load %7 : !cc.ptr<f64>)
